# Trabajar con datos

In [ ]:
# mostrar info de azure-ai-ml
! pip show azure-ai-ml

## Conectar a workspace

In [ ]:
# conectar
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential

ml_client = MLClient.from_config(credential=DefaultAzureCredential())

print(f"Conectado al Workspace: {ml_client.workspace_name}")


## Listar los almacenes de datos

In [ ]:
# listar los almacenes
stores = ml_client.datastores.list()

for aS in stores:
    print(aS.name, ':\n', aS)

## Crear un datastore

To create a datastore and connect to a (already existing) storage, you'll need to specify:

- La clase para indicar con qué tipo de servicio de almacenamiento desea conectar.El siguiente ejemplo se conecta a un almacenamiento Blob (Azure Blob DataStore).
- name: El nombre del almacén de datos en el espacio de trabajo de Azure Machine Learning.
- description: Descripción opcional para proporcionar más información sobre el almacén de datos.
- account_name: El nombre de la cuenta de almacenamiento de Azure.
- container_name: El nombre del contenedor para almacenar blobs en la cuenta de almacenamiento de Azure.
- credentials: Proporcione el método de autenticación y las credenciales para autenticar.El siguiente ejemplo usa una clave de cuenta.

Reemplazar XXXX-XXXX con la clave de cuenta del storage account.

In [ ]:
from azure.ai.ml.entities import AzureBlobDatastore
from azure.ai.ml.entities import AccountKeyConfiguration

store = AzureBlobDatastore(
    name="blob_training_data",
    description="Blob Storage for training data",
    account_name="YOUR-STORAGE-ACCOUNT-NAME",
    container_name="training-data", 
    credentials=AccountKeyConfiguration(
        account_key="XXXX-XXXX"
    ),
)

ml_client.create_or_update(store)

Listar los almacenes de datos nuevamente para verificar que se haya creado un nuevo almacén de datos llamado `blob_training_data`:

In [ ]:
# listar los almacenes
stores = ml_client.datastores.list()

for aS in stores:
    print(aS.name)

## Crear data assets

- `URI_FILE` apunta a un archivo específico.
- `URI_FOLDER` apunta a una carpeta específica.
- `MLTABLE` señala un archivo MLTable que especifica cómo leer uno o más archivos dentro de una carpeta.


In [ ]:
# URI_FILE
from azure.ai.ml.entities import Data
from azure.ai.ml.constants import AssetTypes

my_path = '/home/azureuser/cloudfiles/code/Users/formacion/azure-machine-learning/1.Introduccion_AML_Pipelines/labs/data/diabetes.csv'

my_data = Data(
    path=my_path,
    type=AssetTypes.URI_FILE,
    description="Activo de datos que apunta a un archivo local, cargado automáticamente en el almacén de datos predeterminado",
    name="diabetes-data-local-ric"
)

ml_client.data.create_or_update(my_data)

In [ ]:
# URI_FOLDER
my_path = 'azureml://datastores/training_data_ric_2/paths/datos'

my_data = Data(
    path=my_path,
    type=AssetTypes.URI_FOLDER,
    description="Activo de datos que apunta a un archivo local, cargado automáticamente en el almacén de datos predeterminado",
    name="diabetes-data-folder-ric"
)

ml_client.data.create_or_update(my_data)

In [ ]:
# MLTABLE
my_path = '/home/azureuser/cloudfiles/code/Users/formacion/azure-machine-learning/1.Introduccion_AML_Pipelines/labs/data/'

my_data = Data(
    path=my_path,
    type=AssetTypes.MLTABLE,
    description="Activo de datos que apunta a un archivo local, cargado automáticamente en el almacén de datos predeterminado",
    name="diabetes-data-mltable-ric"
)

ml_client.data.create_or_update(my_data)

Para verificar que se hayan creado los nuevos activos de datos, se puede enumerar todos los activos de datos en el espacio de trabajo nuevamente:

In [ ]:
datasets = ml_client.data.list()
for ds_name in datasets:
    print(ds_name.name)

In [ ]:
# Acceder al Data Asset específico
data_asset = ml_client.data.get(name="diabetes-data-local-ric", version="1")
print(f"Ruta del Data Asset: {data_asset.path}")

## Leer datos en notebook

- Importar `mltable`, luego convertir el activo de datos en un dataframe y visualizarlo.

In [ ]:
# leer mtable
import mltable
from azure.ai.ml import MLClient

data_asset = ml_client.data.get("diabetes-data-mltable-ric", version="1")

tbl = mltable.load(data_asset.path)

df = tbl.to_pandas_dataframe()
df

## Usar datos en un job

In [ ]:
import os

# Crea una carpeta para los archivos de script
script_folder = 'src'
os.makedirs(script_folder, exist_ok=True)
print(script_folder, 'carpeta creada')

In [ ]:
%%writefile /home/azureuser/cloudfiles/code/Users/formacion/azure-machine-learning/1.Introduccion_AML_Pipelines/labs/src/move-data.py
# import libraries
import argparse
import pandas as pd
import numpy as np
from pathlib import Path


def main(args):
    # read data
    df = get_data(args.input_data)

    output_df = df.to_csv((Path(args.output_datastore) / "diabetes.csv"), index = False)


# function that reads the data
def get_data(path):
    df = pd.read_csv(path)

    # Count the rows and print the result
    row_count = (len(df))
    print('Analyzing {} rows of data'.format(row_count))
    
    return df

def parse_args():
    # setup arg parser
    parser = argparse.ArgumentParser()

    # add arguments
    parser.add_argument("--input_data", dest='input_data',
                        type=str)
    parser.add_argument("--output_datastore", dest='output_datastore',
                        type=str)

    # parse args
    args = parser.parse_args()

    print('reading from:', args.input_data)
    print('moving to:', args.output_datastore)

    # return args
    return args

# run script
if __name__ == "__main__":
    # add space in logs
    print("\n\n")
    print("*" * 60)

    # parse args
    args = parse_args()

    # run main function
    main(args)

    # add space in logs
    print("*" * 60)
    print("\n\n")


Enviar un job que ejecute el script **move-data.py**

In [26]:
from azure.ai.ml import Input, Output
from azure.ai.ml.constants import AssetTypes, InputOutputModes
from azure.ai.ml import command

# configure input and output
my_job_inputs = {
    "local_data": Input(type=AssetTypes.URI_FILE, path="azureml:diabetes-data-local-ric:1")
}

my_job_outputs = {
    "datastore_data": Output(type=AssetTypes.URI_FOLDER, path='azureml://datastores/training_data_ric_2/paths/datos', mode=InputOutputModes.RW_MOUNT)
}

# configure job
job = command(
    code="/home/azureuser/cloudfiles/code/Users/formacion/azure-machine-learning/1.Introduccion_AML_Pipelines/labs/src",
    command="python move-data.py --input_data ${{inputs.local_data}} --output_datastore ${{outputs.datastore_data}}",
    inputs=my_job_inputs,
    outputs=my_job_outputs,
    environment="azureml://registries/azureml/environments/sklearn-1.1/versions/4",
    compute="ricardoenm",
    display_name="move-diabetes-data-ric",
    experiment_name="move-diabetes-data-ric"
)

# submit job
returned_job = ml_client.create_or_update(job)
aml_url = returned_job.studio_url
print("Monitor your job at", aml_url)

pathOnCompute is not a known attribute of class <class 'azure.ai.ml._restclient.v2023_04_01_preview.models._models_py3.UriFolderJobOutput'> and will be ignored


Monitor your job at https://ml.azure.com/runs/careful_energy_xg9lz32l9m?wsid=/subscriptions/7decb7a4-f615-4cc3-9d7d-5de10998373f/resourcegroups/naturgy/workspaces/naturgyml4&tid=8b85184d-f6f0-4ace-80dd-c3bda5a6c156


# Ejemplo copia de datasets

In [27]:
from azure.ai.ml import command, Input, Output, MLClient
from azure.ai.ml.constants import AssetTypes, InputOutputModes
from azure.identity import DefaultAzureCredential


input_path = "wasbs://data@azuremlexampledata.blob.core.windows.net/titanic.csv"
output_path = "azureml://datastores/training_data_ric_2/paths/datos/titanic.csv"

data_type = AssetTypes.URI_FILE

input_mode = InputOutputModes.RO_MOUNT

output_mode = InputOutputModes.RW_MOUNT

# Set the input and output for the job:
inputs = {
    "input_data": Input(type=data_type, path=input_path, mode=input_mode)
}

outputs = {
    "output_data": Output(type=data_type, 
                          path=output_path, 
                          mode=output_mode,
                          name = "diabetes_asset"
                  )
}

# This command job copies the data to your default Datastore
job = command(
    command="cp ${{inputs.input_data}} ${{outputs.output_data}}",
    inputs=inputs,
    outputs=outputs,
    environment="azureml://registries/azureml/environments/sklearn-1.1/versions/4",
    compute="ricardoenm",
)

# Submit the command
ml_client.jobs.create_or_update(job)

pathOnCompute is not a known attribute of class <class 'azure.ai.ml._restclient.v2023_04_01_preview.models._models_py3.UriFileJobOutput'> and will be ignored


Experiment,Name,Type,Status,Details Page
code,tidy_guitar_969pfzd03h,command,Starting,Link to Azure Machine Learning studio
